# API Nakala

In [ ]:
# ANR experts
# author: @sardinecan
# date: 2022-12
# description: this Julia notebook allows to interact with the Nakala's API
# licence: CC-0

# @todo : gestion des erreurs (réponses server) ?
# @todo : écrire un fichier de log pour récupérer les identifiants des ressources crées.

## Packages

In [ ]:
using CSV
using DataFrames
using HTTP
using JSON
using Dates

## Identifiants

In [ ]:
path = @__DIR__
credentials = CSV.read(joinpath(path, "credentials.csv"), DataFrame, header=1) #liste des utilisateurs
user = "tnakala" #choix de l'utilisateur (api test = tnakala)
usrCredentials = filter(:user => n -> n == user, credentials) #récupération des identifiants
apiKey = usrCredentials[1, :apikey] #clé API

## API
Pour travailler avec l'API test, modifiez la valeur de la variable `apitest` par `true`

In [ ]:
apitest = true

if apitest == true
  apiurl = "https://apitest.nakala.fr"
else 
  apiurl = "https://api.nakala.fr"
end

## Collections

### Création d'une collection

Une fois les identifiants chargés et l'API sélectionnée, entrez simplement un nom de collection pour la créer.

In [ ]:
collectionName = ""

url = joinpath(apiurl, "collections")

headers = Dict(
  "X-API-KEY" => apiKey,
  "Content-Type" => "application/json"
)

body = Dict(
  :status => "private",
  :metas =>  [Dict(
    :value => collectionName,
    :propertyUri => "http://nakala.fr/terms#title",
    :typeUri => "http://www.w3.org/2001/XMLSchema#string",
    :lang => "fr"
  )]
)

postCollection = HTTP.request("POST", url, headers, JSON.json(body)) # envoi des données pour la création de la collection
collectionResponse = JSON.parse(String(HTTP.payload(postCollection))) # réponse du server
collectionId = collectionResponse["payload"]["id"] # récupération de l'id de la collection
print("Identifiant collection : ", collectionId)

### Ajouter des données à une collection

In [ ]:
collectionIdentifier = "10.34847/nkl.5c519k06"
datas = [
  "nakalaDataId_1",
  "nakalaDataId_2",
  "nakalaDataId_3"
]

url = joinpath(apiurl, "collections", collectionIdentifier, "datas")

headers = Dict(
  "X-API-KEY" => apiKey,
  "Content-Type" => "application/json"
)

body = datas

postDatasToCollection = HTTP.request("POST", url, headers, JSON.json(body)) # ajoute les données listées ci-dessus à une collection
response = JSON.parse(String(HTTP.payload(postDatasToCollection))) # réponse du server

### Supprimer des données d'une collection

In [ ]:
collectionIdentifier = ""
datas = [
  "nakalaDataId_1",
  "nakalaDataId_2",
  "nakalaDataId_3"
]

url = joinpath(apiurl, "collections", collectionIdentifier, "datas")

headers = Dict(
  "X-API-KEY" => apiKey,
  "Content-Type" => "application/json"
)

body = datas

removeDatasFromCollection = HTTP.request("DELETE", url, headers, JSON.json(body)) # supprime les données listées plus haut de la collection
response = JSON.parse(String(HTTP.payload(removeDatasFromCollection))) # réponse du server

## Utilisateurs

### Informations utilisateur

In [ ]:
url = joinpath(apiurl, "users", "me")

headers = Dict(
  "X-API-KEY" => apiKey,
  "Content-Type" => "application/json"
)

userInfo = HTTP.request("GET", url, headers)
userInfoResponse = JSON.parse(String(HTTP.payload(userInfo))) # réponse du server
username = userInfoResponse["username"]
userGroupId = userInfoResponse["userGroupId"]
userInfoResponse

### Ressources utilisateur

In [ ]:
scope = ["deposited", "owned", "shared", "editable", "readable", "all"]
#deposited : les données déposées par l'utilisateur (ROLE_DEPOSITOR)
#owned : les données dont l'utilisateur est propriétaire (ROLE_OWNER)
#shared : les données partagées avec l'utilisateur (ROLE_ADMIN, ROLE_EDITOR ou ROLE_READER, mais pas ROLE_OWNER)
#editable : les données modifiables par l'utilisateur (ROLE_OWNER, ROLE_ADMIN ou ROLE_EDITOR)
#readable : les données lisibles par l'utilisateur (ROLE_OWNER, ROLE_ADMIN, ROLE_EDITOR ou ROLE_READER)
#all : toute
s = scope[1]

url = joinpath(apiurl, "users", "datas", s)

headers = Dict(
  "X-API-KEY" => apiKey,
  "Content-Type" => "application/json"
)

#=
body = Dict(
  :page => 1,
  :limit => 100,
  :orders => [
    "creDate,desc"
  ],
  :types: => [
    "http://purl.org/coar/resource_type/c_c513"
  ],
  :status => [
    "published"
  ],
  :createdYears => [
    "2023"
  ],
  :collections => [
    "11280/9f85fbd6"
  ],
  :titleSearch => "",
  :titleSearchLang => "fr",
  :orderLang => "fr"
)
=#

body = Dict(
  :page => 1,
  :limit => 250
)

userDatas = HTTP.request("POST", url, headers, JSON.json(body))
userDatasResponse = JSON.parse(String(HTTP.payload(userDatas))) # réponse du server

### Lister les données privées/publiques

In [ ]:
scope = ["deposited", "owned", "shared", "editable", "readable", "all"]
#deposited : les données déposées par l'utilisateur (ROLE_DEPOSITOR)
#owned : les données dont l'utilisateur est propriétaire (ROLE_OWNER)
#shared : les données partagées avec l'utilisateur (ROLE_ADMIN, ROLE_EDITOR ou ROLE_READER, mais pas ROLE_OWNER)
#editable : les données modifiables par l'utilisateur (ROLE_OWNER, ROLE_ADMIN ou ROLE_EDITOR)
#readable : les données lisibles par l'utilisateur (ROLE_OWNER, ROLE_ADMIN, ROLE_EDITOR ou ROLE_READER)
#all : toute
s = scope[1]

status = [ "pending" ] #[ "pending", "published" ]

url = joinpath(apiurl, "users", "datas", s)

headers = Dict(
  "X-API-KEY" => apiKey,
  "Content-Type" => "application/json"
)

body = Dict(
  :page => 1,
  :limit => 300,
  :status => status
)

userDatas = HTTP.request("POST", url, headers, JSON.json(body))
userDatasResponse = JSON.parse(String(HTTP.payload(userDatas))) # réponse du server

# récupération des données
datas = get(userDatasResponse, "data", "nothing")

pendingList = [get(data, "identifier", "") for data in datas]

### Publier les données privées

In [ ]:
# nécessite l'exécution de la cellule précédente.

for pendingData in pendingList
  headers = Dict(
    "X-API-KEY" => apiKey,
    "Content-Type" => "application/json"
  )
  
  url = joinpath(apiurl, "datas", pendingData, "status", "published")
  publishData = HTTP.put(url, headers)
  #publishDataResponse = JSON.parse(String(HTTP.payload(publishData))) # réponse du server 
end

### Lister les données

In [ ]:
scope = ["deposited", "owned", "shared", "editable", "readable", "all"]
#deposited : les données déposées par l'utilisateur (ROLE_DEPOSITOR)
#owned : les données dont l'utilisateur est propriétaire (ROLE_OWNER)
#shared : les données partagées avec l'utilisateur (ROLE_ADMIN, ROLE_EDITOR ou ROLE_READER, mais pas ROLE_OWNER)
#editable : les données modifiables par l'utilisateur (ROLE_OWNER, ROLE_ADMIN ou ROLE_EDITOR)
#readable : les données lisibles par l'utilisateur (ROLE_OWNER, ROLE_ADMIN, ROLE_EDITOR ou ROLE_READER)
#all : toute
s = scope[1]

url = joinpath(apiurl, "users", "datas", s)

headers = Dict(
  "X-API-KEY" => apiKey,
  "Content-Type" => "application/json"
)

#=
body = Dict(
  :page => 1,
  :limit => 100,
  :orders => [
    "creDate,desc"
  ],
  :types: => [
    "http://purl.org/coar/resource_type/c_c513"
  ],
  :status => [
    "published"
  ],
  :createdYears => [
    "2023"
  ],
  :collections => [
    "11280/9f85fbd6"
  ],
  :titleSearch => "",
  :titleSearchLang => "fr",
  :orderLang => "fr"
)
=#
# par défaut 100 premiers résultats
body = Dict(
  :page => 1,
  :limit => 500
)

userDatas = HTTP.request("POST", url, headers, JSON.json(body))
userDatasResponse = JSON.parse(String(HTTP.payload(userDatas))) # réponse du server

datas = get(userDatasResponse, "data", "nothing")

list = Vector()

for data in datas
  
  identifier = get(data, "identifier", "")
  metas = get(data, "metas", "")
  push!(m, metas)
  title = filter(x -> get(x, "propertyUri", "") == "http://nakala.fr/terms#title", metas)[1]

  item = Dict(
    get(title, "value", "noTitle") => identifier
  )
  push!(list, item)
end

list

### Chercher une donnée précise

In [ ]:
title = "Z1J432"

scope = ["deposited", "owned", "shared", "editable", "readable", "all"]
#deposited : les données déposées par l'utilisateur (ROLE_DEPOSITOR)
#owned : les données dont l'utilisateur est propriétaire (ROLE_OWNER)
#shared : les données partagées avec l'utilisateur (ROLE_ADMIN, ROLE_EDITOR ou ROLE_READER, mais pas ROLE_OWNER)
#editable : les données modifiables par l'utilisateur (ROLE_OWNER, ROLE_ADMIN ou ROLE_EDITOR)
#readable : les données lisibles par l'utilisateur (ROLE_OWNER, ROLE_ADMIN, ROLE_EDITOR ou ROLE_READER)
#all : toute
s = scope[1]


url = joinpath(apiurl, "users", "datas", s)

headers = Dict(
  "X-API-KEY" => apiKey,
  "Content-Type" => "application/json"
)

body = Dict(
  :page => 1,
  :limit => 100,
  :titleSearch => title,
  :titleSearchLang => "fr",
)

userDatas = HTTP.request("POST", url, headers, JSON.json(body))
userDatasResponse = JSON.parse(String(HTTP.payload(userDatas))) # réponse du server

datas = get(userDatasResponse, "data", "nothing")
# il peut y avoir plusieurs résultats
datas[1]

### Chercher un fichier précis dans une donnée

In [ ]:
title = "Z1J432"
filenames = ["Z1J432_0004.JPG", "Z1J432_0005.JPG", "Z1J432_0006.JPG"]

scope = ["deposited", "owned", "shared", "editable", "readable", "all"]
#deposited : les données déposées par l'utilisateur (ROLE_DEPOSITOR)
#owned : les données dont l'utilisateur est propriétaire (ROLE_OWNER)
#shared : les données partagées avec l'utilisateur (ROLE_ADMIN, ROLE_EDITOR ou ROLE_READER, mais pas ROLE_OWNER)
#editable : les données modifiables par l'utilisateur (ROLE_OWNER, ROLE_ADMIN ou ROLE_EDITOR)
#readable : les données lisibles par l'utilisateur (ROLE_OWNER, ROLE_ADMIN, ROLE_EDITOR ou ROLE_READER)
#all : toute
s = scope[1]


url = joinpath(apiurl, "users", "datas", s)

headers = Dict(
  "X-API-KEY" => apiKey,
  "Content-Type" => "application/json"
)

body = Dict(
  :page => 1,
  :limit => 100,
  :titleSearch => title,
  :titleSearchLang => "fr",
)

userDatas = HTTP.request("POST", url, headers, JSON.json(body))
userDatasResponse = JSON.parse(String(HTTP.payload(userDatas))) # réponse du server

datas = get(userDatasResponse, "data", "nothing")
# il peut y avoir plusieurs résultats
identifier = get(datas[1], "identifier", "")
filesList = get(datas[1], "files", "")

urls = Vector()
for file in filenames
  item = filter(x -> get(x, "name", "") == file, filesList)[1]
  fileIdentifier = get(item, "sha1", "")
  
  fileUrl = joinpath(apiurl, "data", identifier, fileIdentifier)
  push!(urls, fileUrl)
end

urls

## Données

### Publier une donnée avec des fichiers

In [ ]:
include("listFile.jl") # un script qui crée, pour chaque sous-dossier, la liste des images à envoyer.
directories
for directory in directories

  files2upload = CSV.read(joinpath(path, directory, "files.csv"), DataFrame, header=1) # fichier de métadonnées 

  #%% Dépôt des fichiers
  files = Vector()
  filesInfo = []
  for (i, row) in enumerate(eachrow(files2upload))
    filename = row[:filename]
    
    println("Envoi du fichier n°", i, " - ", filename)

    headers = Dict(
      "X-API-KEY" => apiKey, 
      :accept => "application/json"
    )
    
    file = open(joinpath(path, directory, filename), "r")
    body = HTTP.Form(Dict(:file => file))

    fileUpload = HTTP.post(filesUrl, headers, body)
    fileResponse = JSON.parse(String(HTTP.payload(fileUpload)))
    fileIdentifier = fileResponse["sha1"]
    println(fileIdentifier)

    push!(files, fileResponse) # récupération de l'identifiant Nakala du fichier (fileIdentifier) pour le dépot des métadonnées et de la ressource
    push!(filesInfo, [filename, fileIdentifier])
  end

  # métadonnées de la ressource
  meta = Vector()

  metadata = CSV.read(joinpath(path, directory, "metadata.csv"), DataFrame, header=1) # fichier de métadonnées 

  metadata[!, :collections][1] !== missing  ? collections = split(metadata[!, :collections][1], ";") : collections = nothing
  authors = split(metadata[!, :authors][1], ";")
  date = metadata[!, :date][1]
  license = metadata[!, :licence][1]
  status = metadata[!, :status][1]
  datatype = metadata[!, :datatype][1]
  description = metadata[!, :description][1]
  metadata[!, :collections][1] !== missing  ? keywords = split(metadata[!, :keywords][1], ";") : keywords = nothing
  metadata[!, :collections][1] !== missing  ? datarights = split(metadata[!, :rights][1], ";") : datarights = nothing
  lang = metadata[!, :lang][1]

  # titre (obligatoire)
  metaTitle = Dict(
    :value => directory,
    :typeUri => "http://www.w3.org/2001/XMLSchema#string",
    :propertyUri => "http://nakala.fr/terms#title",
    :lang => lang

  )
  push!(meta, metaTitle)

  # datatype (obligatoire)
  metaType = Dict(
    :value => datatype,
    :typeUri => "http://www.w3.org/2001/XMLSchema#anyURI",
    :propertyUri => "http://nakala.fr/terms#type"
  )
  push!(meta, metaType)

  # authorité/creator (obligatoire, mais accepte la valeur null)
  for author in authors   
    if length(split(author, ",")) > 1
      identity = split(author, ",")
      metaAuthor = Dict(
        :value => Dict(
          :givenname => identity[2],
          :surname => identity[1]
        ),
        :propertyUri => "http://nakala.fr/terms#creator"
      )
      push!(meta, metaAuthor)
    else
      metaAuthor = Dict(
        :value => Dict(
          :givenname => author,
          :surname => author
        ),
        :propertyUri => "http://nakala.fr/terms#creator"
      )
      push!(meta, metaAuthor)
    end
  end

  # date (obligatoire, mais accepte la valeur null)    
  metaCreated = Dict(
    :value => Dates.today(),
    :typeUri => "http://www.w3.org/2001/XMLSchema#string",
    :propertyUri => "http://nakala.fr/terms#created"
  )
  push!(meta, metaCreated)
    
  # licence (obligatoire pour une donnée publiée)
  metaLicense = Dict(
    :value => license,
    :typeUri => "http://www.w3.org/2001/XMLSchema#string",
    :propertyUri => "http://nakala.fr/terms#license"
  )
  push!(meta, metaLicense)

  # Droits (facultatif)
  rights = []
  if datarights !== nothing
    for dataright in datarights
      right = Dict(
        :id => split(dataright, ",")[1],
        :role => split(dataright, ",")[2]
      )
      push!(rights, right)
    end
  end

  # Description (facultatif)
  metaDescription = Dict(
    :value => description,
    :lang => lang,
    :typeUri => "http://www.w3.org/2001/XMLSchema#string",
    :propertyUri => "http://purl.org/dc/terms/description"
  )
  push!(meta, metaDescription)

  # Mots-clés
  if keywords !== nothing
    for keyword in keywords
      metaKeyword = Dict(
        :value => keyword,
        :lang => lang,
        :typeUri => "http://www.w3.org/2001/XMLSchema#string",
        :propertyUri => "http://purl.org/dc/terms/subject"
      )
      push!(meta, metaKeyword)
    end
  end

  # assemblage des métadonnées avant envoi de la ressource
  postdata = Dict(
    :collectionsIds => collections,
    :status => "pending",
    :files => files,
    :metas => meta,
    :rights => rights
  )
  println(JSON.json(postdata))

  headers = Dict(
    "X-API-KEY" => apiKey,
    "Content-Type" => "application/json"
  )
   
  metadataUpload = HTTP.request("POST", metadataUrl, headers, JSON.json(postdata))
  metadataResponse = JSON.parse(String(HTTP.payload(metadataUpload))) # réponse du server
  metadataId = metadataResponse["payload"]["id"] # récupération de l'identifiant Nakala de la ressource (identifier)
    
  println(metadataId)

  if isfile(joinpath(path, "datasUploaded.csv"))
    f = open(joinpath(path, "datasUploaded.csv"), "a")       
      write(f, "\n"*directory*","*metadataId)
    close(f)      
  else
    touch(joinpath(path, "datasUploaded.csv"))
    f = open(joinpath(path, "datasUploaded.csv"), "w") 
      write(f, "ressource,identifiant")
      write(f, "\n"*directory*","*metadataId)
    close(f)
  end

  
  touch(joinpath(path, directory, directory*".csv"))
  f = open(joinpath(path, directory, directory*".csv"), "w") 
    write(f, "filename,identifier,fileIdentifier")
    for file in filesInfo
      write(f, "\n"*file[1]*","*metadataId*","*file[2])
    end
  close(f)
end